In [1]:
from utils.libraries import *
from utils.objects import *
from utils.functions import *
from definitions import *
from Economic_Market_Forecasting_ML import *

In [2]:
EMF_ml = Economic_Market_Forecasting()

In [3]:
ROOT_PATH = Path(ROOT_PATH)
DATA_PATH = r"C:\Users\Joan Oliver\Documents\GitHub\EMF_project\data"
config_paths = [
    "config\Market_Data_Config.csv",
    "config\Economic_Data_Config.csv",
    "config\Calc_Data_Config.csv"
]
market_config = ROOT_PATH.joinpath(config_paths[0])
economic_config = ROOT_PATH.joinpath(config_paths[1]).abspath()
calc_config = ROOT_PATH.joinpath(config_paths[2]).abspath()

target_list = pd.read_csv(market_config, sep=";", header=0).loc[:, "Market"].to_list()
# target_list = ['SP500', 'Gold', 'CorporateBonds']


model_pred = {}
y_pred = {}
y_test = {}
top_model = {}
full_test = {}


for target in target_list:
    print(">>Target: "+target)
    model_pred[target], y_pred[target], y_test[target], top_model[target], full_test[target] = EMF_ml.Economic_Market_Forecasting_ML(years=20, 
                                                                                                                                     period="W", 
                                                                                                                                     root_path=ROOT_PATH, 
                                                                                                                                     target=target, 
                                                                                                                                     cutoff_date="2023-01-01", 
                                                                                                                                     cross_val=5, 
                                                                                                                                     data_path=DATA_PATH)

>>Target: SP500
> Load the market data config
> Load the economic data config
> Setting up the indicator dictionaries
> Extracting the indicator data
   -->Extracting 10-Year Treasury Yield...
   -->Extracting 2-Year Treasury Yield...
   -->Extracting Federal Funds Effective Rate...
   -->Extracting 1-Month Treasury Yield...
   -->Extracting 3-Month Treasury Yield...
   -->Extracting 5-Year Treasury Yield...
   -->Extracting 20-Year Treasury Yield...
   -->Extracting 30-Year Treasury Yield...
   -->Extracting 15-Year Mortgage Rate...
   -->Extracting 30-Year Mortgage Rate...
   -->Extracting Unemployment Rate...
   -->Extracting GDP...
   -->Extracting Real GDP...
   -->Extracting CPI...
   -->Extracting PPI...
   -->Extracting Consumer Confidence Index...
   -->Extracting Government Debt...
   -->Extracting Debt Service vs Disposable Income...
   -->Extracting US Dollar Index...
   -->Extracting Corporate Profits...
   -->Extracting EBIT...
   -->Extracting Retained Earnings...
   -->

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv(r'C:\Users\Joan Oliver\Documents\GitHub\Economic_Market_Forecasting\Economic_Market_Forecasting\EMF_webapp\EMF_project\data\SP500_data.csv', sep=";")
data.set_index("Date", inplace=True)
data.index = pd.to_datetime(data.index)
data = data.cumsum().resample("M").last()
# data = data["SP500_Prediction"]
data.index = data.index
data["Date"] = data.index
labels = data["Date"].to_list()
default_items = data["SP500_Prediction"].to_list()
actual_items = data["SP500"].to_list()
data = {
    "labels": labels,
    "default": default_items,
    "actual": actual_items
}